In [8]:
using LinearAlgebra, Plots, Random, Distributions

In [66]:
# structural_params = [alpha, beta, lambda, benefits, beta_hat]
Params = Vector{Float64}([0.0009, 0.995, 0.009, 0.1, 0.9941045])
# grid_sizes = [hc_grid_size, search_grid_size, wage_grid_size]
Sizes = Vector{Int64}([201, 41, 41])
# grid_bounds = [hc_min, hc_max, search_min, search_max, wage_min, wage_max]
Bounds = Vector{Float64}([1.0, 2.0, 0.0, 1.0, 0.0, 1.0])
# human_capital_losses = [unemployment_loss, job_loss]
Losses = Vector{Int64}([10, 30])

2-element Vector{Int64}:
 10
 30

In [34]:
function search_grid(S,B)

    grid = range(B[3], stop=B[4], length=S[2])

    return grid

end

search_grid (generic function with 2 methods)

In [65]:
function wage_grid(S,B)

    grid = range(B[5], stop=B[6], length=S[3])

    lower, upper = 0, 1

    dist = Normal(0.5, 0.1)

    truncated_dist = Truncated(dist, lower, upper)
    # Compute the cumulative distribution function at each point in the wage grid
    cdf_values = [cdf(truncated_dist, point) for point in grid]
    # Compute the probabilities by taking the differences between consecutive CDF values
    probs = diff(cdf_values)

    return grid, probs

end

wage_grid (generic function with 2 methods)

In [36]:
w_grid, w_probs = wage_grid(Sizes, Bounds)

(0.0:0.025:1.0, [7.304320894485256e-7, 2.380591246961792e-6, 7.2908568300754895e-6, 2.098272808764946e-5, 5.6746075900387655e-5, 0.00014421187651184335, 0.0003443961607986427, 0.0007728734323300979, 0.00162986613783184, 0.003229903942435662  …  0.003229903942435719, 0.0016298661378318924, 0.0007728734323300124, 0.00034439616079873225, 0.00014421187651181278, 5.674607590033798e-5, 2.098272808770396e-5, 7.29085683004449e-6, 2.3805912470198365e-6, 7.30432089479649e-7])

In [37]:
function human_capital_grid(S,B)

    grid = range(B[1], stop=B[2], length=S[1])

    return grid

end

human_capital_grid (generic function with 2 methods)

In [38]:
function initial_value_functions(S)

    U = zeros(S[1], 1)

    W = zeros(S[1], S[2])

    return U, W

end

initial_value_functions (generic function with 2 methods)

In [39]:
function cost_of_search(S,B)

    grid = search_grid(S,B)

    cost = (1/2) .* grid 

    return cost

end

cost_of_search (generic function with 1 method)

In [45]:
function contact_probability(S,B)

    grid = search_grid(S,B)

    contact_probs = grid .^ (0.3)

    return contact_probs

end

contact_probability (generic function with 1 method)

Work on Bellman iteration here

In [94]:
function Bellman(B, L, S, P, tolerance, max_iterations, tau)

    U_init, W_init = initial_value_functions(S)

    U_0, W_0 = similar(U_init), similar(W_init)

    store_U_values = zeros(S[2], S[3])

    hc_grid = human_capital_grid(S,B)

    s_grid = search_grid(S,B)

    w_grid, w_probs = wage_grid(S,B)

    cost = cost_of_search(S,B)

    contact_probs = contact_probability(S,B)

    error = tolerance + 1

    iter = 1

    min_hc, max_hc = B[1], B[2]

    b = P[4]

    while error > tolerance

        for i in 1:S[1]

            grid_point = hc_grid[i]

            next_grid_point = hc_grid[i+1]

            hc_tomorrow_unemp = maximum( min_hc, grid_point - ( L[1] * (next_grid_point - grid_point) ) )

            index = findfirst(isequal(hc_tomorrow_unemp), hc_grid)

            for j in 1:S[3]

                wage_index = findfirst(isequal(w_grid[j]), w_grid)

                store_U_values[i,j] = ((1-tau)*b) .- cost .+ (P[5] .* contact_probs) .* [maximum(W_0[index, wage_index], U_0[index])] .+ ((P[5] .* (1 .- contact_probs)) .* U_0[index])

            end

        end

        error = toler - 1.2
    
    end

    return store_U_values

end

Bellman (generic function with 1 method)

In [95]:
out = Bellman(Bounds, Losses, Sizes, Params, 1e-2, 1000, 0.1)

MethodError: MethodError: objects of type Float64 are not callable
Maybe you forgot to use an operator such as *, ^, %, / etc. ?